<div class="alert alert-block alert-info" style="margin-top: 20px">
 <a href="http://cocl.us/DA0101EN_NotbookLink_Top"><img src = "https://ibm.box.com/shared/static/fvp89yz8uzmr5q6bs6wnguxbf8x91z35.png" width = 750, align = "center"></a>
 <h1 align=center><font size = 5> Link</font></h1> 

 <a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Data Analysis with Python</font></h1>

# Module 5: Model Evaluation and Refinement / Modelo de evaluación y refinamiento

Hemos construido modelos y realizado predicciones de precios de vehículos. Ahora determinaremos cuán precisas son estas predicciones.


# Table of content
<p></p>
<li><a href="#ref1">Model Evaluation </a></li>
<li><a href="#ref2">Over-fitting, Under-fitting and Model Selection </a></li>
<li><a href="#ref3">Ridge Regression </a></li>
<li><a href="#ref4">Grid Search</a></li>
<p></p>

In [ ]:
import pandas as pd
import numpy as np

# Import clean data 
path = path='https://ibm.box.com/shared/static/q6iiqb1pd7wo8r3q28jvgsrprzezjqk3.csv'
df = pd.read_csv(path)

Primero solo usemos datos numéricos

In [ ]:
df=df._get_numeric_data()

 Libraries for plotting / Bibliotecas para trazar

In [ ]:
from IPython.display import display
from IPython.html import widgets 
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
print("done")

## Functions for plotting 

In [ ]:
def DistributionPlot(RedFunction,BlueFunction,RedName,BlueName,Title ):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()
    

In [ ]:
def PollyPlot(xtrain,xtest,y_train,y_test,lr,poly_transform):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))
    
    
    #training data 
    #testing data 
    # lr:  linear regression object 
    #poly_transform:  polynomial transformation object 
 
    xmax=max([xtrain.values.max(),xtest.values.max()])

    xmin=min([xtrain.values.min(),xtest.values.min()])

    x=np.arange(xmin,xmax,0.1)


    plt.plot(xtrain,y_train,'ro',label='Training Data')
    plt.plot(xtest,y_test,'go',label='Test Data')
    plt.plot(x,lr.predict(poly_transform.fit_transform(x.reshape(-1,1))),label='Predicted Function')
    plt.ylim([-10000,60000])
    plt.ylabel('Price')
    plt.legend()


<a id="ref1"></a>

# Part 1: Training and Testing / Entrenamiento y Pruebas

Un paso importante para probar su modelo es dividir sus datos en datos de entrenamiento y prueba (**training and testing**). Colocaremos los datos objetivo **precio** en un marco de datos separado **y**:


In [ ]:
y_data=df['price']

drop price data in x data / caída de datos de precios en x datos

In [ ]:
x_data=df.drop('price',axis=1)

ahora dividimos aleatoriamente nuestros datos en datos de entrenamiento y prueba usando la función **train_test_split**


In [ ]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=1)


print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])


El parámetro **test_size** establece la proporción de datos que se divide en el conjunto de prueba. En lo anterior, el conjunto de pruebas se establece en el 10% del conjunto de datos total.



 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #1): </h1>

<b> Use the function "train_test_split" to split up the data set such that 40% of the data samples will be utilized for testing, set the parameter "random_state" equal to zero. The output of the function should be the following:  "x_train_1" , "x_test_1", "y_train_1" and  "y_test_1".</b>
</div>

 <div align="right">
<a href="#q1" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q1" class="collapse">
```
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_data, y_data, test_size=0.4, random_state=0) 
```
</div>



Importemos **LinearRegression** desde el módulo **linear_model**


In [ ]:
from sklearn.linear_model import LinearRegression

Creamos un objeto de regresión lineal:

In [ ]:
lre=LinearRegression()

ajustamos el modelo usando la función horsepower 

In [ ]:
lre.fit(x_train[['horsepower']],y_train)

Calculamos el R^2 en los datos de prueba(**test**):

In [ ]:
lre.score(x_test[['horsepower']],y_test)

Podemos ver que R^2 es mucho más pequeño usando los datos de prueba.

In [ ]:
lre.score(x_train[['horsepower']],y_train)

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #2): </h1>
<b> 
Find the R^2  on the test data using 90% of the data for training data
</b>
</div>

 <div align="right">
<a href="#q2" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q2" class="collapse">
```
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_data, y_data, test_size=0.9, random_state=0)
lre.fit(x_train1[['horsepower']],y_train1)
lre.score(x_test1[['horsepower']],y_test1)
```
</div>


A veces no tiene suficientes datos de prueba; Como resultado, es posible que desee realizar una validación cruzada(**Cross-validation**). Repasemos varios métodos que puede usar para la validación cruzada(**Cross-validation**).



## Cross-validation Score / Puntuación de validación cruzada

Permite importar **model_selection** desde el módulo **cross_val_scor**


In [ ]:
from sklearn.model_selection import cross_val_score
print("done")

Ingresamos el objeto, la característica en este caso 'horsepower', los datos de destino (y_data). El parámetro 'cv' determina el número de pliegues; en este caso 4.


In [ ]:
Rcross=cross_val_score(lre,x_data[['horsepower']], y_data,cv=4)

La puntuación predeterminada es R^2; cada elemento en la matriz tiene el valor R^2 promedio en el pliegue:


In [ ]:
Rcross

Podemos calcular el promedio y la desviación estándar de nuestra estimación:

In [ ]:
print("The mean of the folds are", Rcross.mean(),"and the standard deviation is" ,Rcross.std())

Podemos utilizar el error al cuadrado negativo como una puntuación configurando la métrica del parámetro 'puntuación'(**'scoring'**) en 'neg_mean_squared_error'.


In [ ]:
-1*cross_val_score(lre,x_data[['horsepower']], y_data,cv=4,scoring='neg_mean_squared_error')

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #3): </h1>
<b> 
Calculate the average R^2 using two folds, find the average R^2 for the second fold utilizing the horsepower as a feature : 
</b>
</div>

<div align="right">
<a href="#q3" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q3" class="collapse">
```
Rc=cross_val_score(lre,x_data[['horsepower']], y_data,cv=2)
Rc[1]
```
</div>


También puede usar la función 'cross_val_predict' para predecir el resultado (**output**). La función divide los datos en el número especificado de pliegues, utilizando un pliegue para obtener una predicción, mientras que el resto de los pliegues se utilizan como datos de prueba. Primero importe la función:


In [ ]:
from sklearn.model_selection import cross_val_predict

Ingresamos el objeto, la característica en este caso **'horsepower'**, los datos de destino **y_data**. El parámetro 'cv' determina el número de pliegues; en este caso 4. Podemos producir una salida:


In [ ]:
yhat=cross_val_predict(lre,x_data[['horsepower']], y_data,cv=4)
yhat[0:5]


<a id="ref2"></a>

# Part 2: Overfitting, Underfitting and Model Selection 

Resulta que los datos de prueba a veces denominados datos fuera de muestra son una medida mucho mejor de qué tan bien se desempeña su modelo en el mundo real. Una razón para esto es el sobreajuste (**overfitting**); repasemos algunos ejemplos. Resulta que estas diferencias son más evidentes en la regresión lineal múltiple y la regresión polinómica, por lo que exploraremos el sobreajuste (**overfitting**) en ese contexto.


Creemos objetos de regresión lineal múltiple y entrenemos el modelo usando **'horsepower'**, **'curb-weight'**, **'engine-size'** y **'highway-mpg'** como características.

In [ ]:
lr=LinearRegression()
lr.fit(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']],y_train)

Prediction using training data:
Predicción utilizando datos de entrenamiento:

In [ ]:
yhat_train=lr.predict(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
yhat_train[0:5]

 Prediction using test data: 
 Predicción utilizando datos de prueba:

In [ ]:
yhat_test=lr.predict(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
yhat_test[0:5]

Realicemos una evaluación del modelo utilizando nuestros datos de entrenamiento y prueba por separado. Primero importamos la biblioteca seaborn y matplotlibb para trazar(**plotting**).


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Examinemos la distribución de los valores pronosticados de los datos de entrenamiento.

In [ ]:
Title='Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution '
DistributionPlot(y_train,yhat_train,"Actual Values (Train)","Predicted Values (Train)",Title)

Figura 1: Gráfico de los valores pronosticados utilizando los datos de entrenamiento en comparación con los datos de entrenamiento (**training data compared to the training data. **). 

Hasta ahora, el modelo parece estar aprendiendo bien del conjunto de datos de entrenamiento. Pero, ¿qué sucede cuando el modelo encuentra nuevos datos del conjunto de datos de prueba? Cuando el modelo genera nuevos valores a partir de los datos de prueba, vemos que la distribución de los valores pronosticados es muy diferente de los valores objetivos reales.

In [ ]:
Title='Distribution  Plot of  Predicted Value Using Test Data vs Data Distribution of Test Data'
DistributionPlot(y_test,yhat_test,"Actual Values (Test)","Predicted Values (Test)",Title)

Figura 2: Gráfico del valor predicho utilizando los datos de prueba en comparación con los datos de prueba.

Comparando la Figura 1 y la Figura 2; Es evidente que la distribución de los datos de prueba en la Figura 1 es mucho mejor para ajustar los datos. Esta diferencia en la Figura 2 es evidente cuando los rangos son de 5000 a 15 000. Aquí es donde la forma de distribución es excepcionalmente diferente. Veamos si la regresión polinómica también exhibe una caída en la precisión de la predicción al analizar el conjunto de datos de prueba.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
print("done")

####  Overfitting / Sobreajuste
el **Overfitting** ocurre cuando el modelo se ajusta al ruido (**noise**), no al proceso subyacente. Por lo tanto, al probar su modelo utilizando el conjunto de pruebas, su modelo no funciona tan bien como lo es el ruido de modelado, no el proceso subyacente que generó la relación. Creemos un modelo polinomial de grado 5.

Usemos el 55% de los datos para las pruebas y el resto para el entrenamiento:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.45, random_state=0)
print("done")

Realizaremos una transformación polinómica de grado 5 en la característica **'horse power'**. 

In [ ]:
pr=PolynomialFeatures(degree=5)
x_train_pr=pr.fit_transform(x_train[['horsepower']])
x_test_pr=pr.fit_transform(x_test[['horsepower']])
pr

Ahora creemos un modelo de regresión lineal "poly" y entrenemos.

In [ ]:
poly=LinearRegression()
poly.fit(x_train_pr,y_train)

 Podemos ver el resultado de nuestro modelo utilizando el método "predecir" (**predict**). luego asigne los valores a "yhat".

In [ ]:
yhat=poly.predict(x_test_pr )
yhat[0:5]

Tomemos los primeros cinco valores pronosticados y compárelos con los objetivos reales.

In [ ]:
print("Predicted values:", yhat[0:4])
print("True values:",y_test[0:4].values)

Utilizaremos la función "PollyPlot" que definimos al comienzo del laboratorio para mostrar los datos de entrenamiento, los datos de prueba y la función predicha.

In [ ]:
PollyPlot(x_train[['horsepower']],x_test[['horsepower']],y_train,y_test,poly,pr)

Figura 4 Un modelo de regresión polinómica, los puntos rojos representan datos de entrenamiento, los puntos verdes representan datos de prueba y la línea azul representa la predicción del modelo.

Vemos que la función estimada parece rastrear los datos, pero alrededor de 200 horsepower, la función comienza a divergir de los puntos de datos.

 R^2 de los datos de entrenamiento:

In [ ]:
poly.score(x_train_pr, y_train)

 R^2 de los datos de prueba:

In [ ]:
poly.score(x_test_pr, y_test)

Vemos que el R^2 para los datos de entrenamiento es 0.5567 mientras que el R^2 en los datos de prueba fue -29.87. Cuanto más bajo es R^2, peor es el modelo, un R^2 negativo es un signo de sobreajuste (**overfitting**).


Veamos cómo cambia R^2 en los datos de prueba para diferentes polinomios de orden y graficamos los resultados:

In [ ]:
Rsqu_test=[]

order=[1,2,3,4]
for n in order:
    pr=PolynomialFeatures(degree=n)
    
    x_train_pr=pr.fit_transform(x_train[['horsepower']])
    
    x_test_pr=pr.fit_transform(x_test[['horsepower']])    
    
    lr.fit(x_train_pr,y_train)
    
    Rsqu_test.append(lr.score(x_test_pr,y_test))

plt.plot(order,Rsqu_test)
plt.xlabel('order')
plt.ylabel('R^2')
plt.title('R^2 Using Test Data')
plt.text(3, 0.75, 'Maximum R^2 ')    

Vemos que el R^2 aumenta gradualmente hasta que se usa un orden de tres polinomios. Entonces el R^2 disminuye dramáticamente a las cuatro.


La siguiente función se utilizará en la siguiente sección; Por favor, ejecute la celda.

In [ ]:
def f(order,test_data):
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=test_data, random_state=0)
    pr=PolynomialFeatures(degree=order)
    x_train_pr=pr.fit_transform(x_train[['horsepower']])
    x_test_pr=pr.fit_transform(x_test[['horsepower']])
    poly=LinearRegression()
    poly.fit(x_train_pr,y_train)
    PollyPlot(x_train[['horsepower']],x_test[['horsepower']],y_train,y_test,poly,pr)


La siguiente interfaz le permite experimentar con diferentes órdenes polinomiales y diferentes cantidades de datos.

In [ ]:
interact(f, order=(0,6,1),test_data=(0.05,0.95,0.05))

 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4a): </h1>

<b> We can perform polynomial transformations with more than one feature. Create a "PolynomialFeatures" object "pr1" of degree two. ? 
?</b>
</div>

 <div align="right">
<a href="#q2a" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q2a" class="collapse">
```
pr1=PolynomialFeatures(degree=2)
```
</div>



<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4b): </h1>

<b> 
 Transform the training and testing samples for the features 'horsepower', 'curb-weight', 'engine-size' and 'highway-mpg'. Hint: use the method "fit_transform" 
?</b>
</div>

 <div align="right">
<a href="#q2b" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q2b" class="collapse">
```
x_train_pr1=pr.fit_transform(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
x_test_pr1=pr.fit_transform(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
```
</div>



<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4c): </h1>

<b> 
How many dimensions does the new feature have? Hint: use the attribute "shape"
</b>
</div>


<div align="right">
<a href="#q2c" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q2c" class="collapse">
```
There are now 15 features: x_train_pr1.shape 
```
</div>


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4d): </h1>

<b> 
Create a linear regression model "poly1" and train the object using the method "fit" using the polynomial features? </b>
</div>


<div align="right">
<a href="#q2d" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q2d" class="collapse">
```
poly1=linear_model.LinearRegression().fit(x_train_pr1,y_train)
```
</div>



 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4e): </h1>

<b> 
 
 Use the method  "predict" to predict an output on the polynomial features, then use the function "DistributionPlot"  to display the distribution of the predicted output vs the test data?</b>
</div>

 <div align="right">
<a href="#q2e" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q2e" class="collapse">
```
yhat_test1=poly1.predict(x_train_pr1)
Title='Distribution  Plot of  Predicted Value Using Test Data vs Data Distribution of Test Data'
DistributionPlot(y_test,yhat_test1,"Actual Values (Test)","Predicted Values (Test)",Title)
```
</div>


 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4f): </h1>

<b> 
 Use the distribution plot to determine the two regions were the predicted prices are less accurate than the actual prices.
 </div>

 <div align="right">
<a href="#q2f" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q2f" class="collapse">
```
The predicted value is lower than actual value for cars where the price  $ 10,000 range, conversely the predicted price is larger than the price cost in the $30, 000 to $40,000 range. As such the model is not as accurate in these ranges  
```
<img src = "https://ibm.box.com/shared/static/c35ipv9zeanu7ynsnppb8gjo2re5ugeg.png" width = 700, align = "center">



</div>


<a id="ref3"></a>

## Part 3: Ridge regression 

En esta sección, revisaremos la Ridge regression  y veremos cómo el parámetro Alfa cambia el modelo. Solo una nota aquí nuestros datos de prueba se utilizarán como datos de validación.

Realicemos una transformación polinómica de grado dos en nuestros datos.

In [ ]:
pr=PolynomialFeatures(degree=2)
x_train_pr=pr.fit_transform(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])
x_test_pr=pr.fit_transform(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])

Importemos **Ridge** desde el módulo **modelos lineales**.

In [ ]:
from sklearn.linear_model import Ridge

Creemos un objeto de regresión Ridge, estableciendo el parámetro de regularización en 0.1 

In [ ]:
RigeModel=Ridge(alpha=0.1)

Al igual que la regresión regular, puede ajustar el modelo utilizando el método **fit**.

In [ ]:
RigeModel.fit(x_train_pr,y_train)

Del mismo modo, puede obtener una predicción:

In [ ]:
yhat=RigeModel.predict(x_test_pr)

Comparemos las primeras cinco muestras predichas con nuestro conjunto de pruebas.

In [ ]:
print('predicted:', yhat[0:4])
print('test set :', y_test[0:4].values)

Seleccionamos el valor de Alfa que minimiza el error de prueba, por ejemplo, podemos usar un bucle for.

In [ ]:
Rsqu_test=[]
Rsqu_train=[]
dummy1=[]
ALFA=5000*np.array(range(0,10000))
for alfa in ALFA:
    RigeModel=Ridge(alpha=alfa) 
    RigeModel.fit(x_train_pr,y_train)
    Rsqu_test.append(RigeModel.score(x_test_pr,y_test))
    Rsqu_train.append(RigeModel.score(x_train_pr,y_train))

Podemos trazar el valor de R^2 para diferentes alfa

In [ ]:
width = 12
height = 10
plt.figure(figsize=(width, height))

plt.plot(ALFA,Rsqu_test,label='validation data  ')
plt.plot(ALFA,Rsqu_train,'r',label='training Data ')
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.legend()


Figura 6: La línea azul representa el R^2 de los datos de prueba y la línea roja representa el R^2 de los datos de entrenamiento. El eje x representa los diferentes valores de Alfa

La línea roja en la figura 6 representa el R^2 de los datos de prueba, a medida que Alpha aumenta, el R^2 disminuye; por lo tanto, a medida que Alfa aumenta, el modelo funciona peor en los datos de prueba. La línea azul representa el R ^ 2 en los datos de validación, a medida que el valor de Alfa aumenta, el R^2 disminuye.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #5): </h1>

Perform Ridge regression and calculate the R^2 using the polynomial features, use the training data to train the model and test data to test the model. The parameter alpha should be set to  10.
</div>

<div align="right">
<a href="#q5" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q5" class="collapse">
```
RigeModel=Ridge(alpha=0) 
RigeModel.fit(x_train_pr,y_train)
RigeModel.score(x_test_pr, y_test)
```
</div>


<a id="ref4"></a>

## Part 4: Grid Search / Búsqueda de cuadrícula

El término Alfa es un hiperparámetro, sklearn tiene la clase **GridSearchCV** para simplificar el proceso de búsqueda del mejor hiperparámetro.


Importemos **GridSearchCV** desde el módulo **model_selection**

In [ ]:
from sklearn.model_selection import GridSearchCV
print("done")

Creamos un diccionario de valores de parámetros:

In [ ]:
parameters1= [{'alpha': [0.001,0.1,1, 10, 100, 1000,10000,100000,100000]}]
parameters1

Create a ridge regions object:
Cree un objeto de regiones de cresta:

In [ ]:
RR=Ridge()
RR

Create a ridge grid search object 
Crear un objeto de búsqueda de cuadrícula de cresta (**ridge**)

In [ ]:
Grid1 = GridSearchCV(RR, parameters1,cv=4)

Ajustar el modelo

In [ ]:
Grid1.fit(x_data[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']],y_data)

El objeto encuentra los mejores valores de parámetro en los datos de validación. Podemos obtener el estimador con los mejores parámetros y asignarlo a la variable BestRR de la siguiente manera:

In [ ]:
BestRR=Grid1.best_estimator_
BestRR

Ahora probamos nuestro modelo en los datos de prueba

In [ ]:
BestRR.score(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']],y_test)

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #6): </h1>
Perform a grid search for the alpha parameter and the normalization parameter, then find the best values of the parameters

</div>

 <div align="right">
<a href="#q6" class="btn btn-default" data-toggle="collapse">Click here for the solution</a>

</div>
<div id="q6" class="collapse">
```
parameters2= [{'alpha': [0.001,0.1,1, 10, 100, 1000,10000,100000,100000],'normalize':[True,False]} ]
Grid2 = GridSearchCV(Ridge(), parameters2,cv=4)
Grid2.fit(x_data[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']],y_data)
Grid2.best_estimator
```
</div>


# About the Authors:  

This notebook written [Joseph Santarcangelo PhD]( https://www.linkedin.com/in/joseph-s-50398b136/)

Copyright &copy; 2017 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).


 <div class="alert alert-block alert-info" style="margin-top: 20px">
 <a href="http://cocl.us/DA0101EN_NotbookLink_bottom"><img src = "https://ibm.box.com/shared/static/cy2mwm7519t4z6dxefjpzgtbpi9p8l7h.png" width = 750, align = "center"></a>
 <h1 align=center><font size = 5> Link</font></h1> 